In [1]:
import pandas as pd
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LinearRegression

In [2]:
def rfefeature(indep_x,dep_y,n):
    rfelist=[]
    lin=LinearRegression()
    RF=RandomForestRegressor(n_estimators = 10, random_state = 0)
    DT=DecisionTreeRegressor(random_state = 0)
    svc_model =SVR(kernel = 'linear')
    
    rfemodellist=[lin,RF,DT,svc_model]
    
    for i in rfemodellist:
        log_rfe = RFE(estimator=i, n_features_to_select=n)
        log_fit = log_rfe.fit(indep_x,dep_y)
        log_rfe_feature=log_fit.transform(indep_x)
        rfelist.append(log_rfe_feature)
    return rfelist

In [3]:
def split_scalar(indep_x,dep_y):
    x_train,x_test,y_train,y_test=train_test_split(indep_x,dep_y,test_size=0.25,random_state=0)
    sc=StandardScaler()
    x_train=sc.fit_transform(x_train)
    x_test=sc.transform(x_test)
    return x_train,x_test,y_train,y_test

def R2_prediction(regressor,x_test,y_test):
    y_pred=regressor.predict(x_test)
    from sklearn.metrics import r2_score
    R2=r2_score(y_test,y_pred)
    return R2

def linear(x_train,y_train,x_test):
    from sklearn.linear_model import LinearRegression
    regressor=LinearRegression()
    regressor.fit(x_train,y_train)
    R2=R2_prediction(regressor,x_test,y_test)
    return R2

def svm_linear(x_train,y_train,x_test):
    regressor=SVR(kernel="linear")
    regressor.fit(x_train,y_train)
    R2=R2_prediction(regressor,x_test,y_test)
    return R2

def svm_NL(x_train,y_train,x_test):
    regressor=SVR(kernel="rbf")
    regressor.fit(x_train,y_train)
    R2=R2_prediction(regressor,x_test,y_test)
    return R2

def DecisionTree(x_train,y_train,x_test):
    from sklearn.tree import DecisionTreeRegressor
    regressor=DecisionTreeRegressor(random_state=0)
    regressor.fit(x_train,y_train)
    R2=R2_prediction(regressor,x_test,y_test)
    return R2

def RandomForest(x_train,y_train,x_test):
    from sklearn.ensemble import RandomForestRegressor
    regressor=RandomForestRegressor(n_estimators=10,random_state=0)
    regressor.fit(x_train,y_train)
    R2=R2_prediction(regressor,x_test,y_test)
    return R2

In [4]:
def rfe_Regression(acclin,accsvm1,accdes,accrf):
    rfedataframe=pd.DataFrame(index=["Linear","SVC","Random","DecisionTree"],columns=["linear","SVM1","Decision","Random"])
    for number,index in enumerate(rfedataframe.index):
        rfedataframe["linear"][index]=acclin[number]
        rfedataframe["SVM1"][index]=accsvm1[number]
        rfedataframe["Decision"][index]=accdes[number]
        rfedataframe["Random"][index]=accrf[number]
    return rfedataframe   

In [5]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df=pd.get_dummies(df2,drop_first=True)
indep_x=df.drop("classification_yes",axis=1)
dep_y=df["classification_yes"]

In [6]:
rfelist=rfefeature(indep_x,dep_y,2)
acclin=[]
accsvm1=[]
accsvmn1=[]
accdes=[]
accrf=[]

In [12]:
for i in rfelist:
    x_train,x_test,y_train,y_test=split_scalar(i,dep_y)

    r2_lin=linear(x_train,y_train,x_test)
    acclin.append(r2_lin)

    r2_sl=svm_linear(x_train,y_train,x_test)
    accsvm1.append(r2_sl)
    
    r2_d=DecisionTree(x_train,y_train,x_test)
    accdes.append(r2_d)

    r2_r=RandomForest(x_train,y_train,x_test)
    accrf.append(r2_r)

    result=rfe_Regression(acclin,accsvm1,accdes,accrf)

In [13]:
result

,linear,SVM1,Decision,Random
Linear,0.154714,-0.292446,0.154714,0.154309
SVC,0.154714,-0.292446,0.154714,0.154309
Random,0.154714,-0.292446,0.154714,0.154309
DecisionTree,0.154714,-0.292446,0.154714,0.154309
